# Introduction to atomman: Basic Support and Analysis Tools

__Lucas M. Hale__, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), _Materials Science and Engineering Division, NIST_.
    
[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 

From https://github.com/usnistgov/atomman/tree/master/doc/tutorial

## 1. Introduction

This Notebook outlines some of the other tools in atomman that provide basic support features and simple analysis of the atomistic systems.

**Library Imports**

In [1]:
# Standard Python libraries
from __future__ import (absolute_import, print_function,
                        division, unicode_literals)
import warnings
warnings.filterwarnings('ignore')
import os
from io import open
from copy import deepcopy
import datetime

# http://www.numpy.org/
import numpy as np

# https://github.com/usnistgov/atomman
import atomman as am
import atomman.unitconvert as uc

# Show atomman version
print('atomman version =', am.__version__)

# Show date of Notebook execution
print('Notebook executed on', datetime.date.today())

atomman version = 1.4.7
Notebook executed on 2023-02-14


Construct a demonstration 2x2x2 diamond cubic silicon system

In [2]:
a = uc.set_in_units(5.431, 'angstrom')
box = am.Box(a=a, b=a, c=a)
pos = [[0.00, 0.00, 0.00], [0.50, 0.50, 0.00], [0.50, 0.00, 0.50], [0.00, 0.50, 0.50],
       [0.25, 0.25, 0.25], [0.75, 0.75, 0.25], [0.75, 0.25, 0.75], [0.25, 0.75, 0.75]]
atoms = am.Atoms(atype=1, pos=pos)
ucell = am.System(atoms=atoms, box=box, scale=True)
system = ucell.supersize(2,2,2)
print(system.natoms)

64


## 2. Elastic constants 

The full elastic constants tensor for a given crystal can be represented with the atomman.ElasticConstants class.  The values in an ElasticConstants object can be set and retrieved in a variety of formats and transformed to other Cartesian coordinate systems. 

See the [03.1. ElasticConstants class Jupyter Notebook](03.1. ElasticConstants class.ipynb) for more details and a full description of all of the class methods.

In [3]:
# Define an ElasticConstants object for diamond cubic silicon
# values taken from http://www.ioffe.ru/SVA/NSM/Semicond/Si/mechanic.html
C11 = uc.set_in_units(16.60 * 10**11, 'dyn/cm^2')
C12 = uc.set_in_units( 6.40 * 10**11, 'dyn/cm^2')
C44 = uc.set_in_units( 7.96 * 10**11, 'dyn/cm^2')

C = am.ElasticConstants(C11=C11, C12=C12, C44=C44)

In [4]:
# Get 6x6 Cij Voigt representation of elastic constants in GPa
print('Cij (GPa) =')
print(uc.get_in_units(C.Cij, 'GPa'))

Cij (GPa) =
[[166.   64.   64.    0.    0.    0. ]
 [ 64.  166.   64.    0.    0.    0. ]
 [ 64.   64.  166.    0.    0.    0. ]
 [  0.    0.    0.   79.6   0.    0. ]
 [  0.    0.    0.    0.   79.6   0. ]
 [  0.    0.    0.    0.    0.   79.6]]


## 3. Relative distances between atoms

There are a few built-in tools for investigating the relative positions between atoms of the same and different systems.

### 3.1. System.dvect()

The System.dvect() method computes the shortest vector(s) between two points or list of points within the atomman.System taking into account the System's periodic dimensions.

Parameters

- **pos_0** (*numpy.ndarray or index*) Absolute Cartesian vector position(s) to use as reference point(s). If the value can be used as an index, then self.atoms.pos[pos_0] is taken.

- **pos_1** (*numpy.ndarray or index*) Absolute Cartesian vector position(s) to find relative to pos_0.  If the value can be used as an index, then self.atoms.pos[pos_1] is taken.

- **code** (*str, optional*) Option for specifying which underlying code function to use. 'cython' uses the version of the function built in cython (faster). 'python' uses the purely python version.  Default is 'cython' if the code can be imported, otherwise 'python'.

In [5]:
# Calculate shortest vector between atoms 1 and 60
print(system.dvect(1, 60))

[ 4.07325  4.07325 -4.07325]


In [6]:
# Calculate shortest distance between position [5., 5., 5.] and all atoms in system
pos = system.atoms.pos

dvects = system.dvect([5.0, 5.0, 5.0], pos)
print(np.linalg.norm(dvects, axis=1))

[8.66025404 5.95297241 5.95297241 5.95297241 6.30856205 3.87088054
 3.87088054 3.87088054 7.08419092 6.33398788 6.33398788 3.25939281
 5.45266877 5.86626957 5.86626957 2.2175116  7.08419092 6.33398788
 3.25939281 6.33398788 5.45266877 5.86626957 2.2175116  5.86626957
 5.03701519 6.69334927 3.91218142 3.91218142 4.43455051 7.33774633
 4.93424363 4.93424363 7.08419092 3.25939281 6.33398788 6.33398788
 5.45266877 2.2175116  5.86626957 5.86626957 5.03701519 3.91218142
 6.69334927 3.91218142 4.43455051 4.93424363 7.33774633 4.93424363
 5.03701519 3.91218142 3.91218142 6.69334927 4.43455051 4.93424363
 4.93424363 7.33774633 0.7465139  4.4706471  4.4706471  4.4706471
 3.09820588 6.6163557  6.6163557  6.6163557 ]


### 3.2. displacement()

The atomman.displacement() function compares two systems with the same number of atoms and calculates the vector differences between all atoms with the same atomic id's. The vectors returned are the shortest vectors after taking periodic boundaries in consideration, i.e. it uses dvect().

Parameters

- **system_0** (*atomman.System*) The initial system to calculate displacements from.

- **system_1** (*atomman.System*) The final system to calculate displacements to.

- **box_reference** (*str or None*) Specifies which system's boundary conditions to use.

    - 'initial' uses system_0's box and pbc.
    
    - 'final' uses system_1's box and pbc (Default).
    
    - None computes the straight difference between the positions without accounting for periodic boundaries.
    
- **code** (*str, optional*) Option for specifying which code version of dvect to use (see dvect's documentation for details).

In [7]:
# Copy system and randomly displace atoms
system2 = deepcopy(system)
system2.atoms.pos += 3 * np.random.rand(system.natoms, 3)
system2.wrap()

# Show displacement between the two systems
print(am.displacement(system, system2))       

[[1.99158133 1.31495772 1.64803024]
 [0.07062241 0.25973257 0.59354138]
 [2.89007324 0.01307383 0.66097233]
 [0.12216748 0.99513805 1.30301415]
 [0.81180696 1.02920297 1.66146325]
 [2.24817424 1.58053854 0.97006709]
 [1.78521106 2.10211383 1.96923951]
 [2.84259265 1.30230631 1.54789338]
 [1.1050786  1.60470442 0.71244999]
 [2.62015497 0.13789465 1.87670035]
 [1.00012491 2.84631954 2.6408456 ]
 [2.17673575 2.51976516 0.50049919]
 [0.97332327 0.26760703 0.56001093]
 [1.63430526 1.13034027 2.02274302]
 [1.91795287 1.17853018 1.48560958]
 [0.91885011 0.06360409 1.93787699]
 [2.42884357 2.88913739 1.36969071]
 [0.57012454 2.64598611 0.6381404 ]
 [0.2105172  0.73465138 1.67329176]
 [2.50495214 2.64848225 1.99228699]
 [1.77996514 2.94071201 0.83630993]
 [2.24808675 0.95793251 0.37390873]
 [1.61529679 0.71181607 1.99212167]
 [1.38095155 1.11777481 0.52598343]
 [1.23287346 2.98181602 2.42108148]
 [2.98536721 0.82642135 0.6994703 ]
 [1.64873049 0.38387573 2.78493768]
 [0.73776296 2.39306666 2.04

### 3.3. System.neighborlist()

A list of neighbor atoms within a cutoff can be constructed using the System.neighborlist() method.  The list of neighbors is returned as an atomman.NeighborList object.

See the [03.2. NeighborList class Jupyter Notebook](03.2. NeighborList class.ipynb) for more details on how the list is calculated and can be used.

Parameters
        
- **cutoff** (*float, optional*) Radial cutoff distance for identifying neighbors.  Must be given if model is not given.

- **model** (*str or file-like object, optional*) Gives the file path or content to load.  If given, no other parameters are allowed.
            
- **cmult** (*int, optional*) Parameter associated with the binning routine.  Default value is most likely the fastest.
            
- **code** (*str, optional*)  Option for specifying which underlying code function to use. 'cython' uses the version of the function built in cython (faster). 'python' uses the purely python version.  Default is 'cython' if the code can be imported, otherwise 'python'.

- **initialsize** (*int, optional*) The number of neighbor positions to initially assign to each atom.  Default value is 20.

- **deltasize** (*int, optional*) Specifies the number of extra neighbor positions to allow each atom when the number of neighbors exceeds the underlying array size.  Default value is 10.
            
Returns
        
- (*atomman.NeighborList*) The compiled list of neighbors.

In [8]:
# Identify neighbors within 3 angstroms
neighbors = system.neighborlist(cutoff=3)

In [9]:
# Show average atomic coordination
print('Average coordination =', neighbors.coord.mean())

Average coordination = 4.0


In [10]:
# List neighbor atoms of atom 6
print('Neighbors of atom 6 =', neighbors[6])

Neighbors of atom 6 = [ 2 11 33 40]


## 4. Basic tools

This lists some of the other basic tools and features in atomman.

### 4.1. Atomic information

- **atomman.tools.atomic_number()** returns the atomic number associated with an element's atomic symbol.  

- **atomman.tools.atomic_symbol()** returns the elemental symbol associated with an given atomic number.

- **atomman.tools.atomic_mass()** returns the atomic mass of an element or isotope. The atom can be identified with atomic number or atomic/isotope symbol.

In [11]:
# Get atomic number for an atomic symbol
num = am.tools.atomic_number('Fe')
print(num)

# Get atomic symbol for an atomic number
symbol = am.tools.atomic_symbol(num)
print(symbol)

# Get atomic mass for an atomic symbol
mass = am.tools.atomic_mass(symbol)
print(mass)

# Get atomic mass for an atomic number
mass = am.tools.atomic_mass(num)
print(mass)

26
Fe
55.845
55.845


In [12]:
# Get atomic mass for an isotope
mass = am.tools.atomic_mass('Al-26')
print(mass)

25.986891904


### 4.2. axes_check()

The axes_check() function is useful when working in Cartesian systems. Given a (3,3) array representing three 3D Cartesian vectors:

- The three vectors are checked that they are orthogonal and right-handed.

- The corresponding array of unit vectors are returned. This can then be used for crystal transformations.

In [13]:
axes = [[-1, 0, 1],
        [ 1, 0, 1],
        [ 0, 1, 0]]
print(am.tools.axes_check(axes))

[[-0.70710678  0.          0.70710678]
 [ 0.70710678  0.          0.70710678]
 [ 0.          1.          0.        ]]


### 4.3. filltemplate()

The filltemplate() function takes a template and fills in values for delimited template variables.

In [14]:
madlibs = "My friend <name> really likes to use templates to <verb>, says that they are <adjective>!"
s_delimiter = '<'
e_delimiter = '>'

terms = {}
terms['name'] = 'Charlie'
terms['verb'] = 'program'
terms['adjective'] = 'delicious'

print(am.tools.filltemplate(madlibs, terms, s_delimiter, e_delimiter))

My friend Charlie really likes to use templates to program, says that they are delicious!


### 4.4. indexstr()

Iterates through all indicies of an array with a given shape, returning both the numeric index and a string representation.

In [15]:
for index, istr in am.tools.indexstr((3,2)):
    print('index ->', repr(index), ', istr ->', repr(istr))

index -> (0, 0) , istr -> '[0][0]'
index -> (0, 1) , istr -> '[0][1]'
index -> (1, 0) , istr -> '[1][0]'
index -> (1, 1) , istr -> '[1][1]'
index -> (2, 0) , istr -> '[2][0]'
index -> (2, 1) , istr -> '[2][1]'


### 4.5. uber_open_rmode

uber_open_rmode is a context manager that allows for similar reading of content from a file or from a string variable. It equivalently handles:
    
- str path name to a file

- str content

- open file-like object

In [16]:
# Define str and save to file
text = 'Here I am, read me!'
fname = 'text.txt'
with open(fname, 'w') as f:
    f.write(text)

# Use uber_open_rmode on text
with am.tools.uber_open_rmode(text) as f:
    print(f.read())
    
# Use uber_open_rmode on file path
with am.tools.uber_open_rmode(fname) as f:
    print(f.read())
    
# Use uber_open_rmode on file-like object
with open(fname, 'rb') as fobject:
    with am.tools.uber_open_rmode(fobject) as f:
        print(f.read())

b'Here I am, read me!'
b'Here I am, read me!'
b'Here I am, read me!'


### 4.6. vect_angle()

The vect_angle() function returns the angle between two vectors.

In [17]:
vect1 = 2*np.random.rand(3)-1
vect2 = 2*np.random.rand(3)-1

print('Angle between', vect1, 'and', vect2, '=')
print(am.tools.vect_angle(vect1, vect2), 'degrees')
print(am.tools.vect_angle(vect1, vect2, 'radian'), 'radians')

Angle between [-0.74381808  0.92348464  0.50491477] and [0.53938015 0.31082861 0.23697373] =
89.63317406255615 degrees
1.5643940064047868 radians


### 4.7. Miller index conversions

- **atomman.tools.miller.vector3to4(indices)** converts three-term Miller indices to four-term Miller-Bravais for hexagonal systems.

- **atomman.tools.miller.vector4to3(indices)** converts four-term Miller-Bravais to three-term Miller indices.

- **atomman.tools.miller.vector_crystal_to_cartesian(indices, box)** converts Miller and Miller-Bravais indices to Cartesian vectors based on a supplied box.

**Note**: The returned indices will be the smallest integer representations possible, i.e. direction is retained but magnitude may be different. 

In [18]:
# Test single value case
print(am.tools.miller.vector3to4(np.array([3,3,3])))
print(am.tools.miller.vector4to3(np.array([1,1,-2,0])))

[ 1.  1. -2.  3.]
[3. 3. 0.]


In [19]:
# Generate random uvw crystal indices
indices = np.random.randint(-5,6, (3,3))
print(indices)
print()

# Convert to hexagonal uvtw's
indices = am.tools.miller.vector3to4(indices)
print(indices)
print()

# Convert back to uvw's
indices = am.tools.miller.vector4to3(indices)
print(indices)

[[-1  5  0]
 [-4 -1  5]
 [-2  5  0]]

[[-2.33333333  3.66666667 -1.33333333  0.        ]
 [-2.33333333  0.66666667  1.66666667  5.        ]
 [-3.          4.         -1.          0.        ]]

[[-1.  5.  0.]
 [-4. -1.  5.]
 [-2.  5.  0.]]


In [20]:
# Define a hexagonal box
a = uc.set_in_units(2.51, 'angstrom')
c = uc.set_in_units(4.07, 'angstrom')
box = am.Box(a=a, b=a, c=c, gamma=120)

# Pass Miller indices
indices = [[1,0,0],
           [0,1,0],
           [0,0,1]]
print(am.tools.miller.vector_crystal_to_cartesian(indices, box))
print()

# Pass equivalent Miller-Bravais indices
indices = [[ 2,-1,-1, 0],
           [-1, 2,-1, 0],
           [ 0, 0, 0, 1]]
print(am.tools.miller.vector_crystal_to_cartesian(indices, box))

[[ 2.51        0.          0.        ]
 [-1.255       2.17372376  0.        ]
 [ 0.          0.          4.07      ]]

[[ 7.53        0.          0.        ]
 [-3.765       6.52117129  0.        ]
 [ 0.          0.          4.07      ]]


### 4.8. Crystal lattice identification

There are also a few tests for identifying if a supplied box is consistent with a standard representation of a crystal family unit cell.

- **atomman.tools.identifyfamily(box)** returns str crystal family if box corresponds to a standard crystal representation. Otherwise, returns None.

- **atomman.tools.iscubic(box))** returns bool indicating if box is a standard cubic box.

- **atomman.tools.ishexagonal(box))** returns bool indicating if box is a standard hexagonal box.

- **atomman.tools.istetragonal(box))** returns bool indicating if box is a standard tetragonal box.
 
- **atomman.tools.isrhombohedral(box))** returns bool indicating if box is a standard rhombohedral box.

- **atomman.tools.isorthorhombic(box))** returns bool indicating if box is a standard orthorhombic box.

- **atomman.tools.ismonoclinic(box))** returns bool indicating if box is a standard monoclinic box.

- **atomman.tools.istriclinic(box))** returns bool indicating if box is a standard triclinic box.

All of these functions use the following standard representation criteria:

- cubic: 
    - $a = b = c$
    - $\alpha = \beta = \gamma = 90$
- hexagonal: 
    - $a = b \ne c$
    - $\alpha = \beta = 90$
    - $\gamma = 120$
- tetragonal: 
    - $a = b \ne c$
    - $\alpha = \beta = \gamma = 90$
- rhombohedral:
    - $a = b = c$
    - $\alpha = \beta = \gamma \ne 90$
- orthorhombic: 
    - $a \ne b \ne c$
    - $\alpha = \beta = \gamma = 90$
- monoclinic: 
    - $a \ne b \ne c$
    - $\alpha = \gamma = 90$
    - $\beta \ne 90$
- triclinic: 
    - $a \ne b \ne c$
    - $\alpha \ne \beta \ne \gamma$

In [21]:
# Define an orthogonal box
a = uc.set_in_units(2.51, 'angstrom')
b = uc.set_in_units(3.13, 'angstrom')
c = uc.set_in_units(4.07, 'angstrom')
box = am.Box(a=a, b=b, c=c)

print('identifyfamily =', am.tools.identifyfamily(box))
print('iscubic =       ', am.tools.iscubic(box))
print('ishexagonal =   ', am.tools.ishexagonal(box))
print('istetragonal =  ', am.tools.istetragonal(box))
print('isrhombohedral =', am.tools.isrhombohedral(box))
print('isorthorhombic =', am.tools.isorthorhombic(box))
print('ismonoclinic =  ', am.tools.ismonoclinic(box))
print('istriclinic =   ', am.tools.istriclinic(box))

identifyfamily = orthorhombic
iscubic =        False
ishexagonal =    False
istetragonal =   False
isrhombohedral = False
isorthorhombic = True
ismonoclinic =   False
istriclinic =    False


In [22]:
# Define a non-standard tetragonal box with a=c!=b
box = am.Box(a=a, b=b, c=a)
print('identifyfamily =', am.tools.identifyfamily(box))

identifyfamily = None


**File Cleanup**

In [23]:
os.remove('text.txt')